In [40]:
import pandas as pd
import re
import numpy as np
import sys

# Chemin du fichier Excel
fichier_excel = '/Users/stevecostalat/Downloads/SAISIE_CODIFRANCE_OP2507_2025-06-21 12_51_16.407839.xlsx'

# Charger la première feuille par défaut
df = pd.read_excel(fichier_excel)
df['RONDE DES MARQUES'] = df['RONDE DES MARQUES'].apply(lambda x : True if str(x).strip() in ['X', 'x'] else False)
for i in range(2,7,1):
    df[f"DESCRIPTIF_{i}"] = df[f"DESCRIPTIF_{i}"].fillna('').astype('str')
    df[f"PRIX AU KG_{i}"] = df[f"PRIX AU KG_{i}"].fillna('').astype('str')
    df[f"PRIX AU KG DU LOT VIRTUEL_{i}"] = df[f"PRIX AU KG DU LOT VIRTUEL_{i}"].fillna('').astype('str')

# Afficher les 5 premières lignes
df['GENCOD'] = df['GENCOD'].astype('int').astype('str')
df.columns

Index(['ISFROM_LOGO', 'CLE', 'CODE OP', 'DATE OP', 'MENTION SPECIFIQUE',
       'CATEGORIE', 'INTITULE', 'GENCOD', 'MARQUE', 'ORIGINE', 'MECAPROMO',
       'RONDE DES MARQUES', 'DESCRIPTIF', 'PRIX AU KG', 'PVC MAXI', 'BONUS',
       'BONUS EN %', 'RI EN €', 'RI EN %', 'PVC NET', 'LOT VIRTUEL',
       'PRIX AU KG DU LOT VIRTUEL', 'FORMAT GV', 'FORMAT MV', 'FORMAT MVK',
       'FORMAT PV', 'PRODUIT DE UNE', 'PRODUIT EN DER', 'MISE EN AVANT',
       'VIDE_2', 'PICTO', 'SR', 'CATALOG', 'CATALOG_MARKET', 'AFFICHE',
       'SELECTION FRANCAP', 'SELECTION CODIFRANCE', 'INFO COMPLEMENTAIRES',
       'PHOTO1', 'PHOTO2', 'PHOTO3', 'PHOTO4', 'PHOTO5', 'PHOTO6', 'PHOTO7',
       'PHOTO8', 'DESCRIPTIF_2', 'PRIX AU KG_2', 'PRIX AU KG DU LOT VIRTUEL_2',
       'DESCRIPTIF_3', 'PRIX AU KG_3', 'PRIX AU KG DU LOT VIRTUEL_3',
       'DESCRIPTIF_4', 'PRIX AU KG_4', 'PRIX AU KG DU LOT VIRTUEL_4',
       'DESCRIPTIF_5', 'PRIX AU KG_5', 'PRIX AU KG DU LOT VIRTUEL_5',
       'DESCRIPTIF_6', 'PRIX AU KG_6', 'P

In [41]:

regex_saveur = r"[oO]u en|[eE]xiste aussi(?: en)?|\b[Oo]u\b"
regex_le_la = r"^(Le|La|L')\s*"
# Amélioration : ignorer les nombres suivis de °, %, ou accolés à des lettres, ou non suivis d'un mot
regex_unitaire = r"\b\d+(?!\s?(cm|g|mg|ml|cl|l)\b|\s*°|\s*%|\w|,)\b"
# Amélioration : accepte aussi les nombres à virgule (ex: 381,9 g)
regex_contenu = r"\b(\d+(?:[.,]\d+)?\s?(cm|g|mg|ml|cl|l))\b"



# Découpe la chaîne en utilisant la regex regex_saveur et aussi les virgules
def split_by_saveur_and_comma(text):
    """
    Découpe une chaîne de saveurs/packagings selon les connecteurs ('ou', 'existe aussi en', etc.) et les virgules,
    puis nettoie chaque élément pour ne garder que les saveurs pertinentes.
    """
    # Découpe d'abord selon les connecteurs de saveurs
    parts = re.split(regex_saveur, text)
    result = []
    for part in parts:
        cleaned = re.sub(regex_saveur, '', part, flags=re.IGNORECASE).strip()
        if not cleaned:
            continue
        # Découpe par virgule, nettoie chaque sous-partie
        for p in cleaned.split(','):
            p_clean = p.strip()
            # Ignore les lignes commençant par Le/La/L' et contenant une taille/unité
            if re.match(regex_le_la, p_clean) and (re.search(regex_contenu, p_clean) or re.search(regex_unitaire, p_clean)):
                continue
            # Si ce n'est pas un packaging, retire la taille/unité éventuelle
            if not re.match(regex_le_la, p_clean):
                p_clean = re.sub(regex_contenu, '', p_clean).strip()
                # Coupe à la première occurrence d'un nombre unitaire
                p_clean = re.split(regex_unitaire, p_clean, maxsplit=1)[0].strip()
            if p_clean:
                result.append(p_clean)
    return result



def parse_descriptif_lines_into_dict(row):
    """
    Parse chaque ligne du champ 'DESCRIPTIF' pour extraire les saveurs, packaging, taille et unité.
    Retourne une liste de dictionnaires structurés.
    """

    results = []
    lines = str(row['DESCRIPTIF']).split('\n')

    for line in lines:
        # Recherche de la taille/unité via regex
        match_c = re.search(regex_contenu, line)
        match_u = re.search(regex_unitaire, line)
        saveurs = split_by_saveur_and_comma(line)

        if match_c or match_u:
            match = match_c if match_c else match_u
            packaging = line[:match.start()].strip()
            size = ''.join(filter(str.isdigit, match.group(0)))
            # Récupère l'unité (lettres) si regex_contenu, sinon tente après le match unitaire
            unit = (
                ''.join(filter(str.isalpha, match.group(0)))
                if match_c else
                (line[match_u.end():].strip().split()[0] if match_u and line[match_u.end():].strip() else None)
            )
            # Retirer packaging, size et unit de la ligne pour misc (version condensée)
            misc = line
            for val in (packaging, size, unit):
                if val:
                    misc = misc.replace(val, '', 1)
            misc = misc.strip()
            results.append({
                'saveurs': saveurs,
                'packaging': packaging if packaging else None,
                'size': size if size else None,
                'unit': unit if unit else None,
                'misc': misc if misc else None
            })
        else:
            if saveurs[0]== line.strip if line.strip() else None:
                # Cas où il n'y a que des saveurs sans taille/unité
                results.append({
                    'saveurs': None,
                    'packaging': None,
                    'size': None,
                    'unit': None,
                    'misc': line.strip() if line.strip() else None
                    })
            # Cas sans taille/unité détectée
            else:
                results.append({
                    'saveurs': saveurs,
                    'packaging': None,
                    'size': None,
                    'unit': None,
                    'misc': None
                })

    return results

def merge_descriptif_dict(d_dict):
    """
    Fusionne une liste de dictionnaires descriptifs en regroupant les saveurs par contenant (size).
    - d_dict: liste de dictionnaires avec les clés 'saveurs', 'packaging', 'size', 'unit'
    Retourne une liste de dictionnaires fusionnés.
    """
    assert d_dict, "DESCRIPTION should not be empty"

    # Vérifier que tous les dictionnaires ont les mêmes unités
    # Met à jour d_dict en place si des unités différentes sont trouvées
    units = [item['unit'] for item in d_dict if item['unit'] is not None]
    if units and any(u != units[-1] for u in units):
        last_unit = units[-1]
        for item in d_dict:
            if item['unit'] is not None and item['unit'] != last_unit:
                # Concaténer packaging, size et unit pour misc
                misc_parts = []
                if item.get('packaging'):
                    misc_parts.append(str(item['packaging']))
                if item.get('size'):
                    misc_parts.append(str(item['size']))
                if item.get('unit'):
                    misc_parts.append(str(item['unit']))
                item['misc'] = " ".join(misc_parts)
                item['packaging'] = None
                item['size'] = None
                item['unit'] = None

    # Définir 'packaging' comme la première valeur non nulle trouvée dans d_dict
    first_packaging = next((item['packaging'] for item in d_dict if item.get('packaging')), None)
    for item in d_dict:
        if not item.get('packaging') and first_packaging:
            item['packaging'] = first_packaging
    # Extraire tous les 'size' non None, en gardant l'ordre et l'unicité
    seen = set()
    contenants = [item['size'] for item in d_dict if item['size'] is not None and not (item['size'] in seen or seen.add(item['size']))]

    result = []
    idx = 0
    while idx < len(d_dict):
        # Pour chaque contenant, regrouper les éléments jusqu'au dernier de ce contenant
        for c in contenants:
            indices_c = [i for i in range(idx, len(d_dict)) if d_dict[i]['size'] == c]
            if not indices_c:
                continue
            last_idx = indices_c[-1]
            # Inclure aussi les éléments sans 'size' avant le dernier de ce contenant
            sub_d_dict = [d_dict[i] for i in range(idx, last_idx + 1) if d_dict[i]['size'] is None or d_dict[i]['size'] == c]
            # Fusionner les saveurs en gardant l'ordre et l'unicité
            saveurs = list(dict.fromkeys(s for item in sub_d_dict for s in item['saveurs']))
            packaging = next((d['packaging'] for d in sub_d_dict if d['packaging']), None)
            unit = next((d['unit'] for d in sub_d_dict if d['size'] == c and d['unit']), None)
            misc = next((d['misc'] for d in sub_d_dict if d['misc']), None)
            result.append({
                'saveurs': saveurs,
                'packaging': packaging,
                'size': c,
                'unit': unit,
                'misc': misc
            })
            idx = last_idx + 1
        break  # On sort après avoir traité tous les contenants

    return result

In [42]:
def apply_sr_process(df):
    df['desc_dict'] = df.apply(lambda row: merge_descriptif_dict(parse_descriptif_lines_into_dict(row)), axis=1)
    # Extraire la liste de toutes les saveurs (hors None ou chaînes vides)
    df['len'] = df['desc_dict'].apply(lambda desc: sum(len(d.get('saveurs', [])) for d in desc))
    df['CLE'] = df['CLE'].astype(int)*100
    df['PRIX AU KG'].astype('object')
    
    units_t = {'g':1000, 'kg':1, 'mg':1000000, 'ml':1000, 'cl':100, 'l':1, 'default': 1}

    #selectionner les conditions de degroupage, SR, et Catalogue:
    isgroup = (~df['RONDE DES MARQUES']) & \
            ((df['PHOTO2'].notna()) | (df['DESCRIPTIF_2'].notna()) | (df['len'] > 1))

    # Retourner la liste des clés (CLE) et des indices du DataFrame filtré
    code = list(zip(df[isgroup]['CLE'], df[isgroup].index))
    for cle, idx in code:
        desc_dict = df.at[idx, 'desc_dict']
        df.at[idx, 'SR'] = ''   # mettre le champs SR à nulle pour la ligne groupée
        df.at[idx, 'CATALOG'] = 'X'  # mettre le champs CATALOG

        px_net = df.at[idx, 'PVC NET']
        px_max = df.at[idx, 'PVC MAXI']

        len_saveurs = sum(len(d.get('saveurs', [])) for d in desc_dict)

        for i, d in enumerate(desc_dict):
            colname = f"DESCRIPTIF{'' if i == 0 else f'_{i + 1}'}"
            value = '\n'.join(filter(None, [
                d.get('misc', ''),
                ', '.join(d.get('saveurs', [])),
                ' '.join(filter(None, [d.get('packaging'), d.get('size'), d.get('unit')]))
            ]))
            df.at[idx, colname] = value  # 👈 Modification directe sans slice ni copie

            # Cherche un nombre entier dans d['packaging'], sinon retourne 1
            match_cond = re.search(r"\b(\d+)\b", str(d.get('packaging', '')))
            conditionnement = int(match_cond.group(1)) if match_cond else 1

            colpkg = f"PRIX AU KG{'' if i==0 else f'_{i + 1}'}"
            value = df.at[idx, "PRIX AU KG"]
            if i> 0:            
                regex_euro = r"\b\d{1,3}(?:[.,]\d{3})*(?:[.,]\d{2})?(?=\s?€)"
                matches = re.findall(regex_euro, value)
                
                p1 = px_net * units_t.get(d['unit'], 1) / (conditionnement * float(d['size'].replace(',', '.')))
                p2 = px_max * units_t.get(d['unit'], 1) / (conditionnement * float(d['size'].replace(',', '.')))
                value = re.sub(matches[0], f"{p1:.2f}".replace(".",","), value)
                if len(matches) > 0:
                    value = re.sub(matches[1], f"{p2:.2f}".replace(".",","), value)
            df.at[idx, colpkg] = value  # 👈 Modification directe sans slice ni copie

            colpkglv = f"PRIX AU KG DU LOT VIRTUEL{'' if i==0 else f'_{i + 1}'}"
            value = df.at[idx, "PRIX AU KG DU LOT VIRTUEL"]
            if i> 0:            
                matches = re.findall(regex_euro, value)
                if len(matches) > 1:
                    value = re.sub(matches[2], f"{p1:.2f}".replace(".",","), value)
            df.at[idx, colpkglv] = value  # 👈 Modification directe sans slice ni copie




            indice=0
            for sav in d.get('saveurs', []):
                indice += 1
                new_row = df.loc[idx].copy()
                new_row['CLE'] = cle+indice
                new_row['DESCRIPTIF'] = '\n'.join(filter(None, [d.get('misc', ''), sav, ' '.join(filter(None, [d.get('packaging'), d.get('size'), d.get('unit')]))]))
                new_row['SR'] = 'X'
                new_row['CATALOG'] = ''
                new_row['PRIX AU KG'] = new_row[colpkg]
                df.loc[len(df)] = new_row
    return df.sort_values("CLE", ascending=True)



data = apply_sr_process(df)
data[f"PHOTO2"] = data[f"PHOTO2"].astype('str')
data = data[(~data['RONDE DES MARQUES']) & ((data['PHOTO2'].notna()) | (data['DESCRIPTIF_2']!='') | (data['len'] > 1))]
data = data[['CLE','SR','CATALOG','GENCOD','INTITULE','MARQUE','DESCRIPTIF','DESCRIPTIF_2','PRIX AU KG', 'PRIX AU KG_2', 'PRIX AU KG DU LOT VIRTUEL', 'PRIX AU KG DU LOT VIRTUEL_2']]

data.head(40)

TypeError: expected string or bytes-like object, got 'float'